# World Model Core
*Sean Steinle, Kiya Aminfar*

This notebook walks through the core aspects of world models, developing crucial pieces of code sequentially. Not that this code isn't meant for scale -- instead, this is for a demonstration of how we developed the code that we did.

## Table of Contents
1. [Collecting Rollout Data](#Collecting-Rollout-Data)
2. [Training the VAE](#Training-the-VAE)
3. [Training the MDN-RNN](#Training-the-MDN-RNN)
    - [Prepping Rollout Data for MDN-RNN](#Prepping-Rollout-Data-for-MDN-RNN)
    - [Core Training](#Core-Training)
4. [Training the Controller](#Training-the-Controller)
5. [Early Results](#Early-Results)

In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
import os
import numpy as np

## Collecting Rollout Data

In [2]:
#Let's begin by creating an instance of our humanoid environment and checking out what basic observations look like.
env = gym.make('Humanoid-v5', render_mode="rgb_array")
obs, info = env.reset()

In [3]:
obs.shape, obs

((348,),
 array([ 1.40481817e+00,  1.00027820e+00, -7.58427598e-03, -5.84756869e-03,
        -6.20780298e-03, -8.03125789e-03,  5.34441239e-04, -9.52304832e-03,
         9.27527694e-03,  6.16362533e-03, -5.64603477e-03, -7.81711139e-04,
        -7.90436194e-03, -9.48778159e-03, -6.75732395e-03, -2.45679302e-03,
         9.02893471e-03, -8.95145457e-03, -2.64243844e-04,  6.57908475e-03,
        -7.28041357e-03, -5.25994481e-03, -6.57503454e-03,  7.37232827e-04,
         3.19486864e-03, -7.50552959e-03,  3.64205754e-03, -2.85553166e-03,
         7.78082012e-03, -7.50665087e-03,  4.38009133e-03, -1.25463233e-03,
        -7.41374027e-03,  8.10338273e-03,  4.99328496e-04, -2.31017587e-04,
         7.73390741e-03, -8.60626718e-03, -4.45227518e-03,  2.46440318e-03,
         6.74017134e-03,  2.83517671e-03, -8.46904104e-03, -6.50741013e-03,
         7.69759353e-04,  2.30114170e+00,  2.28639280e+00,  4.69434572e-02,
         1.55334553e-03,  1.02410907e-01, -3.88335083e-02, -2.13578796e-01,
   

In [4]:
info

{'x_position': np.float64(-0.001028115287979985),
 'y_position': np.float64(0.0018464669009018685),
 'tendon_length': array([0.00430053, 0.00486432]),
 'tendon_velocity': array([ 0.00415399, -0.00760405]),
 'distance_from_origin': np.float64(0.0021134003552342653)}

As we can see, the humanoid environment gives us a TON of observations! We get dozens of variables representing various positions and velocities of body parts, the center of mass, and a lot of other variables I hardly understand. For an exhaustive list, see the [doc](https://gymnasium.farama.org/environments/mujoco/humanoid/#observation-space). The fact that there are so many variables here is what makes learning latent observations so obvious!

We also get some nice summary stats in info, but we aren't going to include them in our scrape.

In [6]:
def collect_rollout_data(env_name: str, out_dir: str, n_timesteps: int=10000, print_n_episodes: int=1000):
    """Simulates `n_timesteps` in the `env_name` environment, saving observations to an .npy file at `out_dir`."""
    env = gym.make(env_name, render_mode='rgb_array')
    obs, info = env.reset()
    observations = []
    episode_count = 0

    for timestep in range(n_timesteps):  # Run for n_timesteps or until the episode ends
        action = env.action_space.sample() #select random action
        obs, reward, terminated, truncated, info = env.step(action) #execute and get results
        observations.append(obs) #save observation
        if terminated or truncated: #check for game over, if so reset env
            episode_count+=1
            if episode_count % print_n_episodes == 0: print(f"finished {episode_count} episodes") #provide update on training
            observation, info = env.reset()
        env.close()
    np_obs = np.array(observations)
    print(f"final observations has shape: {np_obs.shape}")
    np.save(f'{out_dir}/{env_name}_{n_timesteps}_rollout_observations.npy', np_obs) #load with: new_obs = np.load("../data/processed/Humanoid-v5_10000_rollout_observations.npy")
    return np_obs

In [7]:
humanoid_obs = collect_rollout_data('Humanoid-v5', "../data/processed", 10000, 100)

finished 100 episodes
finished 200 episodes
finished 300 episodes
finished 400 episodes
final observations has shape: (10000, 348)


## Training the VAE

Now that we have an easy function for gathering observations, we need to train the VAE module of our world model which will compress the observation space into latent space with fewer dimensions.

Note that the original World Model implementation worked with tensorflow 1.18.0. This is incredibly outdated (worked with Python 3.5), so let's get a newer version (tensorflow 2.19.0). Additionally, we need to change the structure of the VAE from working with images to working with a vector of observation data! Luckily, ChatGPT is very good at updating code (or it will be very obvious if it is not!).

In [30]:
import tensorflow as tf
from tensorflow.keras import layers, Model, saving

@saving.register_keras_serializable()
class MLPVAE(Model):
    def __init__(self, input_dim=348, z_size=32, kl_tolerance=0.5):
        super(MLPVAE, self).__init__()
        self.z_size = z_size
        self.kl_tolerance = kl_tolerance

        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(2 * z_size),  # output both mu and logvar
        ])

        # Decoder
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(z_size,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim, activation='linear'),  # output same shape as input
        ])

    def sample_z(self, mu, logvar):
        eps = tf.random.normal(shape=tf.shape(mu))
        sigma = tf.exp(0.5 * logvar)
        return mu + sigma * eps

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = tf.split(h, num_or_size_splits=2, axis=1)
        logvar = tf.clip_by_value(logvar, -10.0, 10.0)  # helps with exploding values
        return mu, logvar

    def decode(self, z):
        return self.decoder(z)

    def call(self, x):
        mu, logvar = self.encode(x)
        z = self.sample_z(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

    def compute_loss(self, x):
        x_recon, mu, logvar = self(x)
        recon_loss = tf.reduce_mean(tf.reduce_sum(tf.square(x - x_recon), axis=1))
        kl_loss = -0.5 * tf.reduce_sum(1 + logvar - tf.square(mu) - tf.exp(logvar), axis=1)
        kl_loss = tf.maximum(kl_loss, self.kl_tolerance * self.z_size)
        kl_loss = tf.reduce_mean(kl_loss)
        total_loss = recon_loss + kl_loss
        return total_loss, recon_loss, kl_loss

Let's also write a training function for convenience.

In [15]:
def create_dataset(x_train, batch_size=64, shuffle_buffer=10000):
    # Assuming x_train is a NumPy array of shape [n_samples, 348]
    dataset = tf.data.Dataset.from_tensor_slices(x_train.astype(np.float32))
    dataset = dataset.shuffle(shuffle_buffer).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def train_vae(model, dataset, epochs=10, learning_rate=1e-4):
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    for epoch in range(epochs):
        total_loss = 0.0
        total_batches = 0
        for x_batch in dataset:
            with tf.GradientTape() as tape:
                loss, recon_loss, kl_loss = model.compute_loss(x_batch)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            total_loss += loss.numpy()
            total_batches += 1

        avg_loss = total_loss / total_batches
        print(f"Epoch {epoch+1}: avg loss = {avg_loss:.4f}")


In [22]:
# x_train should be a NumPy array of shape (n_samples, 348)
x_train = humanoid_obs
x_train = (x_train - np.mean(x_train, axis=0)) / (np.std(x_train, axis=0) + 1e-6)

dataset = create_dataset(humanoid_obs, batch_size=64)
vae = MLPVAE(input_dim=348, z_size=32)
train_vae(vae, dataset, epochs=20)

Epoch 1: avg loss = 289430.8125
Epoch 2: avg loss = 119881.2266
Epoch 3: avg loss = 68145.3984
Epoch 4: avg loss = 56953.0703
Epoch 5: avg loss = 51048.7578


2025-05-06 21:55:19.295242: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 6: avg loss = 44877.3750
Epoch 7: avg loss = 38613.2305
Epoch 8: avg loss = 33449.9492
Epoch 9: avg loss = 29071.9258
Epoch 10: avg loss = 24597.8613
Epoch 11: avg loss = 21002.9824
Epoch 12: avg loss = 18676.8027
Epoch 13: avg loss = 17292.6426
Epoch 14: avg loss = 15747.7051
Epoch 15: avg loss = 14375.2168
Epoch 16: avg loss = 13334.0107
Epoch 17: avg loss = 12652.3604
Epoch 18: avg loss = 11696.5762
Epoch 19: avg loss = 11132.5664
Epoch 20: avg loss = 10717.4766


Loss is going down! At first I got a tons of NAN values, but it's because I wasn't normalizing the input data and I also needed to clip the logvar values we were getting as a result of the encoding process. If you get NANs again, a lower learning rate could help too. Onto saving the model!

In [36]:
vae.save_weights('../models/vae/humanoid_10000_vae_model.weights.h5') #save ONLY weights -- much simpler than serializing the entire object

In [37]:
new_vae = MLPVAE(input_dim=348, z_size=32) #instantiate new model object 
new_vae(tf.zeros((1, 348))) #invoke it to build its shape 
new_vae.load_weights('../models/vae/humanoid_10000_vae_model.weights.h5') #now load weights into empty vector

In [38]:
new_vae

<MLPVAE name=mlpvae_4, built=True>

In [39]:
vae

<MLPVAE name=mlpvae_3, built=True>

## Training the MDN-RNN